In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 20 06:54:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 90.5 MB/s eta 0:00:00


In [ ]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.5 MB/s eta 0:00:00


In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Kaggle')
os.chdir('otto')

In [ ]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

## data -> score

0:10683577 -> 0.547

10683577:21367154 -> 0.548

21367154:32050731 -> 0.548

32050731:42734308 -> 0.548

42734308:53417885 -> 0.548

53417885:64101462 -> 0.548

64101462:74785039 -> 0.548

74785039:85468616 -> 0.547

85468616:96152193 -> 0.547

96152193:106835770 -> 0.547

106835770:117519347 -> 0.547

117519347:128202924 -> 0.547

128202924:138886501 -> 0.548

138886501:149570078 -> 0.548

149570078:160253655 -> 0.547

160253655:170937232 -> 0.544

170937232:181620809 -> 0.54

181620809:192304386 -> 0.54

192304386:202987963 -> 0.54

202987963:213671540 -> 0.54

In [ ]:
# train = pl.read_parquet('./OTTO_train_and_validation/train.parquet')
#train.sample(n=1000, random_state=0).reset_index(drop=True)
start_idx = 10683577 * 17
end_idx = 10683577 * 18
train = pl.read_parquet('./OTTO_train_and_validation/train.parquet')[start_idx:end_idx]
train_sub = pl.read_parquet('./OTTO_train_and_validation/test.parquet')
test = pl.read_parquet('./otto_full_optimized_memory_footprint/test.parquet')

In [ ]:
test_label = pl.read_parquet('./OTTO_train_and_validation/test_labels.parquet')
test_label.head()

session,type,ground_truth
i64,str,list[i64]
11098528,"""clicks""",[1679529]
11098528,"""carts""",[1199737]
11098528,"""orders""","[990658, 950341, ... 1033148]"
11098529,"""clicks""",[1105029]
11098530,"""orders""",[409236]


In [ ]:
train.head()

session,aid,ts,type
i32,i32,i32,u8


In [ ]:
train_sub.head()

session,aid,ts,type
i32,i32,i32,u8
11098528,11830,1661119200,0
11098529,1105029,1661119200,0
11098530,264500,1661119200,0
11098530,264500,1661119288,0
11098530,409236,1661119369,0


In [ ]:
# ((163955180, 4), (7683577, 4), (6928123, 4))
train.shape, train_sub.shape, test.shape

((0, 4), (7683577, 4), (6928123, 4))

In [ ]:
# df = pl.concat([train, test])
df = pl.concat([train, train_sub, test])
#df = pl.read_parquet('../input/otto-train-and-test-data-for-local-validation/test.parquet')

df = df.sort(['session', 'aid', 'ts'])
df = df.with_columns((pl.col('ts') * 1e9).alias('ts'))
df = df.rename({'session': 'session:token', 'aid': 'aid:token', 'ts': 'ts:float'})

In [ ]:
!mkdir recbox_data
df['session:token', 'aid:token', 'ts:float'].write_csv('./recbox_data/recbox_data.inter', sep='\t')

del df
gc.collect()

mkdir: cannot create directory ‘recbox_data’: File exists


In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk

In [ ]:
MAX_ITEM = 30

parameter_dict = {
    'data_path': '/content/drive/MyDrive/Kaggle/otto/',
    'USER_ID_FIELD': 'session',
    'ITEM_ID_FIELD': 'aid',
    'TIME_FIELD': 'ts',
    'user_inter_num_interval': "[5,inf)",
    'item_inter_num_interval': "[5,inf)",
    'load_col': {'inter': ['session', 'aid', 'ts']},
    'train_neg_sample_args': None,
    'epochs': 10,
    'stopping_step':3,
    'eval_batch_size': 1024,
    'MAX_ITEM_LIST_LENGTH': MAX_ITEM,
    'eval_args': {
        'split': {'RS': [9, 1, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}

config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

In [ ]:
# https://qiita.com/fufufukakaka/items/e03df3a7299b2b8f99cf
from typing import List, Tuple
import numpy as np
import torch

from pydantic import BaseModel
from recbole.data import create_dataset
from recbole.data.dataset.sequential_dataset import SequentialDataset
from recbole.data.interaction import Interaction
from recbole.model.sequential_recommender.sine import SINE
from recbole.utils import get_model, init_seed

class ItemHistory(BaseModel):
    sequence: List[str]
    topk: int

class RecommendedItems(BaseModel):
    score_list: List[float]
    item_list: List[str]


def pred_user_to_item(item_history: ItemHistory):
    item_history_dict = item_history.dict()
    item_sequence = item_history_dict["sequence"]
    item_length = len(item_sequence)
    pad_length = MAX_ITEM  # pre-defined by recbole

    padded_item_sequence = torch.nn.functional.pad(
        torch.tensor(dataset.token2id(dataset.iid_field, item_sequence)),
        (0, pad_length - item_length),
        "constant",
        0,
    )

    input_interaction = Interaction(
        {
            "aid_list": padded_item_sequence.reshape(1, -1),
            "item_length": torch.tensor([item_length]),
        }
    )
    scores = model.full_sort_predict(input_interaction.to(model.device))
    scores = scores.view(-1, dataset.item_num)
    scores[:, 0] = -np.inf  # pad item score -> -inf
    topk_score, topk_iid_list = torch.topk(scores, item_history_dict["topk"])

    predicted_score_list = topk_score.tolist()[0]
    predicted_item_list = dataset.id2token(
        dataset.iid_field, topk_iid_list.tolist()
    ).tolist()

    recommended_items = {
        "score_list": predicted_score_list,
        "item_list": predicted_item_list,
    }
    return recommended_items

In [ ]:
#test = pl.read_parquet('../input/otto-train-and-test-data-for-local-validation/test.parquet')
test = pl.read_parquet('./otto_full_optimized_memory_footprint/test.parquet')

import pandas as pd
import numpy as np

from collections import defaultdict

sample_sub = pd.read_csv('./sample_submission.csv')

session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

labels = []

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        AIDs = list(dict.fromkeys(AIDs))
        item = ItemHistory(sequence=AIDs, topk=20)

        try:
            nns = pred_user_to_item(item)['item_list']
            labels.append((AIDs+nns)[:20])
        except:
            labels.append(AIDs)

/usr/local/lib/python3.8/dist-packages/recbole/data/dataset/dataset.py:1376: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.field2id_token[field][ids]


In [ ]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

prediction_dfs = []

for st in session_types:
    modified_predictions = predictions.copy()
    modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(modified_predictions)

submission = pd.concat(prediction_dfs).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)